In [4]:
import sys
import pandas as pd
sys.path.insert(1, '/Users/max/projects/machine-learning-labs/tensorflow')
from mytensorflow import *
import tensorflow as tf
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow_datasets as tfdt
import pandas as pd
import numpy as np
import os
from keras.layers import Dense, Flatten, Reshape, Input, InputLayer
from keras.models import Sequential, Model

from  matplotlib import pyplot as plt
import random
import tensorflow as tf
import os
from PIL import Image
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
from tensorflow import keras
import matplotlib.image as mpimg
%matplotlib inline

/Users/max/projects/machine-learning-labs/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
train_ds, test_ds = tf.keras.utils.image_dataset_from_directory(
    "../data/catsdogs",
    validation_split=0.2, 
    subset="both", 
    seed=42,
    image_size=(200,200), 
    batch_size=32, 
)

Found 2000 files belonging to 2 classes.
Using 1600 files for training.
Using 400 files for validation.


In [54]:
x_train = np.array(list(train_ds.map(lambda image, label: image).as_numpy_iterator())).reshape(1600, 200,200,3)/255
y_train = np.array(list(train_ds.map(lambda image, label: label).as_numpy_iterator())).reshape(1600,-1)
print(x_train[:1])

[[[[0.56078434 0.56078434 0.52156866]
   [0.56078434 0.56078434 0.52156866]
   [0.56345093 0.56345093 0.52423525]
   ...
   [0.         0.01176471 0.04705882]
   [0.         0.01176471 0.04705882]
   [0.         0.01176471 0.04705882]]

  [[0.564549   0.564549   0.5178039 ]
   [0.56078434 0.56078434 0.5140392 ]
   [0.564549   0.564549   0.52533334]
   ...
   [0.         0.008      0.05458824]
   [0.         0.008      0.05458824]
   [0.         0.008      0.05458824]]

  [[0.56941175 0.5623529  0.52235293]
   [0.56941175 0.5623529  0.52235293]
   [0.5717647  0.5647059  0.5278432 ]
   ...
   [0.00235294 0.00392157 0.05098039]
   [0.         0.00392157 0.05098039]
   [0.         0.00392157 0.05098039]]

  ...

  [[0.7372549  0.7521568  0.7701961 ]
   [0.74352956 0.7584315  0.7764707 ]
   [0.73647064 0.7513726  0.7694118 ]
   ...
   [0.77254903 0.7866667  0.8015686 ]
   [0.7725489  0.7866666  0.8015685 ]
   [0.7780392  0.7921569  0.80705875]]

  [[0.73380405 0.7534119  0.76909816]
   [0.7

In [65]:
net = Network()
net.add(ConvolutionalLayer((200,200,3), (3,3), 1))
net.add(FlattenLayer())
net.add(FCLayer(39204, 3))
net.add(ActivationLayer(TangH.callFunction, TangH.callDerivative))
net.add(FCLayer(3, 1))
net.add(ActivationLayer(Sigmoid.callFunction, Sigmoid.callDerivative))
net.compile(MeanSquaredError.fun, MeanSquaredError.derivative)
net.fit(x_train[:500], y_train[:500], epochs=2, learning_rate=0.1)

epoch 1/2   error=0.256645
epoch 2/2   error=0.256329


In [66]:
predictions = net.predict(x_train[1000:1300])

In [67]:
for y_pred, y_true in zip(predictions, y_train[1000:1300]):
    print(f"pred: {(y_pred)}, true: {y_true}")

pred: [0.47681629], true: 1
pred: [0.47681629], true: 0
pred: [0.47681629], true: 1
pred: [0.47681629], true: 0
pred: [0.47681629], true: 0
pred: [0.47681629], true: 0
pred: [0.47681629], true: 0
pred: [0.47681629], true: 0
pred: [0.47681629], true: 0
pred: [0.47681629], true: 0
pred: [0.47681629], true: 1
pred: [0.47681629], true: 0
pred: [0.47681629], true: 1
pred: [0.47681629], true: 0
pred: [0.47681629], true: 0
pred: [0.47681629], true: 1
pred: [0.47681629], true: 0
pred: [0.47681629], true: 0
pred: [0.47681629], true: 0
pred: [0.47681629], true: 0
pred: [0.47681629], true: 1
pred: [0.47681629], true: 0
pred: [0.47681629], true: 1
pred: [0.47681629], true: 1
pred: [0.47681629], true: 0
pred: [0.47681629], true: 1
pred: [0.47681629], true: 1
pred: [0.47681629], true: 0
pred: [0.47681629], true: 0
pred: [0.47681629], true: 1
pred: [0.47681629], true: 1
pred: [0.47681629], true: 1
pred: [0.47681629], true: 1
pred: [0.47681629], true: 0
pred: [0.47681629], true: 1
pred: [0.47681629], 

In [5]:
inputs = tf.keras.Input(shape=(200, 200, 3))
x = tf.keras.layers.Rescaling(1.0 / 255)(inputs)
x = tf.keras.layers.Conv2D(64, 3, strides=2, padding="same")(x)
x = tf.keras.layers.MaxPooling2D(3, strides=2, padding="same")(x)
x = tf.keras.layers.Conv2D(128, 4, strides=2, padding="same")(x)
x = tf.keras.layers.MaxPooling2D(3, strides=2, padding="same")(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64, activation="relu")(x)
outputs = tf.keras.layers.Dense(2, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [10]:
history = model.fit(PIL_img_data[::20], epochs=10, validation_data=PIL_img_data[::20])

ValueError: Data is expected to be in format `x`, `(x,)`, `(x, y)`, or `(x, y, sample_weight)`, found: (array([[[0.37254903, 0.3764706 , 0.3137255 ],
        [0.35686275, 0.36078432, 0.29803923],
        [0.34901962, 0.3529412 , 0.2901961 ],
        ...,
        [0.70980394, 0.6392157 , 0.54509807],
        [0.69803923, 0.63529414, 0.54509807],
        [0.6784314 , 0.6156863 , 0.5254902 ]],

       [[0.6901961 , 0.6313726 , 0.5568628 ],
        [0.6509804 , 0.5921569 , 0.5176471 ],
        [0.81960785, 0.76862746, 0.7058824 ],
        ...,
        [0.47843137, 0.3764706 , 0.31764707],
        [0.4862745 , 0.38431373, 0.3254902 ],
        [0.5019608 , 0.4117647 , 0.34901962]],

       [[0.5176471 , 0.4509804 , 0.37254903],
        [0.38039216, 0.38431373, 0.32156864],
        [0.36862746, 0.37254903, 0.30980393],
        ...,
        [0.654902  , 0.58431375, 0.49019608],
        [0.65882355, 0.59607846, 0.49803922],
        [0.6627451 , 0.6       , 0.50980395]],

       ...,

       [[0